In [ ]:
head -n 5 ~/.zshrc
tail -n 10 ~/.zshrc
# 해당 파일의 위 5 줄 , 아래 10줄 보여주는 코드
head -n 10 ~/.zshrc | tail -n 5 
# 위의 10줄 중 아래 5줄 보여주는 코드, 중간 | 기호를 파이프라인이라고 함
conda list | grep numpy # 콘다 설치된 패키지 중에 넘파이만 출력해줌
history | tail -n 10 # 식으로 응용 가능
!5793 # 이후 느낌표 숫자로 이전 명령어 사용할 수 있다
df -h # 파일 시스템에 있는 사용 가능한 저장 공간을 확인해볼 수 있다

In [ ]:
# select
select CountryCode as Code, Name, Population
, CONCAT(Name, "(", Population, ")") as city_population
from city
where Population between 5000000 and 8000000
-- and CountryCode like "%R%"
-- and (CountryCode = "BRA" or CountryCode = "GBR") # 괄호 치면 괄호 내 부분부터 먼저 처리
-- and CountryCode in ("BRA", "GBR")
order by CountryCode ASC, population DESC
limit 5, 3 # 5번째부터 3개

# group by
select CountryCode, count(CountryCode)
, avg(Population) as avg_population
from city
where Population >= 5000000
group by CountryCode
having avg_population >= 8000000

In [ ]:
# 인덱스, 특정 조건으로 검색할 때 어떻게 하면 빠르게 검색할 수 있을지

In [ ]:
# 우분투 접속 이후
cd python3/database/
mysql -u root -p
create database sakila; # 사킬라 데이터베이스 추가
quit
cd python3/database/
mysql -u root -p sakila < sakila-schema.sql # 스키마 생성
# 한 뒤 리프레쉬 해주면 데이터베이스에 사킬라 생성돼 있음
mysql -u root -p sakila < sakila-data.sql # 데이터 넣기

In [ ]:
# 같은 방식으로
create database employees;
use employees; # Database changed
source employees.sql; # 파일 내에 있는 sql 쿼리문이 실행된다, 파일 이동됨

In [ ]:
# 소수점 올림, 반올림, 버림 함수
select ceil(12.345) # 올림 함수
select round(12.345, 2) # 반올림
select truncate(12.345, 2) # 버림

select Code, round((GNP / Population) * 1000, 2)
from country

# 조건문

# IF : IF(조건, 참, 거짓)
# 도시의 인구가 100만 이상 big city, 100만 미만 small city 출력
# column : city_scale
select name, population
, IF(population >= 1000000, "big city", "small city")
as city_scale
from city

# IFNULL : IFNULL(참, 거짓)
# county 테이블에서 독립년도(IndepYear)가 없으면 0으로 출력
select Name, IFNULL(IndepYear, 0) as IndepYear
from country

# CASE
# CASE
# 		WHEN (조건1) THEN (출력1)
# 		WHEN (조건2) THEN (출력2)
# END AS (컬럼명)

# 나라별 10억 이상, 1억 이상, 1억 이하 조건을 출력
select name, population,
CASE
		WHEN population > 100000000 THEN "upper 1 billion"
		WHEN population > 10000000 THEN "upper 100 million"
		ELSE "below 100 million"
END as result
from country

# DATE_FORMAT : 날짜 데이터의 포맷을 변경해주는 함수
# sakila
use sakila

# payment에서 월별 총 매출 출력
select DATE_FORMAT(payment_date, "%Y-%m") as monthly # 데이트 포맷은 원래 있던 데이트의 포맷을 바꿔주는 역할
, sum(amount), count(amount)
from payment
group by monthly

select DATE_FORMAT(payment_date, "%H") as monthly # 시간대별 매출 분석, %d는 일자별, %W는 요일별
, sum(amount), count(amount)
from payment
group by monthly

# JOIN : merge()와 같은 것
# create table & data
CREATE TABLE user (
user_id int(11) unsigned NOT NULL AUTO_INCREMENT, name varchar(30) DEFAULT NULL,
PRIMARY KEY (user_id)
) ENGINE=InnoDB DEFAULT CHARSET=utf8;

CREATE TABLE addr (
id int(11) unsigned NOT NULL AUTO_INCREMENT, addr varchar(30) DEFAULT NULL,
user_id int(11) DEFAULT NULL,
PRIMARY KEY (id)
) ENGINE=InnoDB DEFAULT CHARSET=utf8;

INSERT INTO user(name)
VALUES ("jin"),
("po"),
("alice"),
("petter");

INSERT INTO addr(addr, user_id)
VALUES ("seoul", 1),
("pusan", 2),
("daejeon", 3), ("deagu", 5), ("seoul", 6);

In [ ]:
# inner join
select addr.addr, addr.user_id, user.name # table이름.컬럼명
from addr
join user
on addr.user_id = user.user_id

# World 도시 이름, 국가 이름을 출력
select city.countrycode, city.name, country.name
from city
join country
on city.countrycode = country.code

# left join
select addr.id, addr.addr, addr.user_id
, IFNULL(user.name, "-") as name
from addr
left join user
on addr.user_id = user.user_id

# right join
select addr.id, addr.addr, addr.user_id, user.name
from addr # 먼저 쓴 쪽이 왼쪽이 된다
right join user
on addr.user_id = user.user_id

# union : select 문의 결과를 합쳐서 출력
# 자동으로 중복을 제거

select name
from user
union # all 붙여주면 중복까지 출력해준다
select addr
from addr

# outer join
# 사실 mysql에는 outer join이라는 것이 없다, union으로 만들어낼 수 있다
select user.name, addr.addr, addr.user_id
from user
left join addr
on user.user_id = addr.user_id
union
select user.name, addr.addr, addr.user_id
from user
right join addr
on user.user_id = addr.user_id

# sub-query : 쿼리 문 안에 쿼리가 있는 문법
# select, from, where 절에서 쓸 수 있다

# 전체 나라 수, 전체 도시 수, 전체 언어 수를 출력
select
(select count(*)
from country) as total_country,
(select count(*)
from city) as total_city,
(select count(distinct(language))
from countrylanguage) as total_language

# 800만 이상이 되는 도시의 국가 코드, 도시 이름, 국가 이름, 도시인구 수를 출력
# 일단 800만 이상이 되는 도시의 국가 코드 출력하는 코드 작성
select *
from
	(select countrycode, name, population
	from city
	where population >= 8000000) as city
join
	(select code, name
	from country) as country
on city.countrycode = country.code

# 800만 이상 도시의 국가코드, 국가 이름, 대통령 이름 출력
select code, name, HeadOfState
from country
where code in (
	select distinct(countrycode)
	from city
	where population >= 8000000
)

In [ ]:
# index : 데이터를 검색할 때 빠르게 찾을 수 있도록 해주는 기능, 색인 목차를 만들어주는 과정
# 인덱스가 있으면 검색이 빠르고 없으면 느리다
use employees

explain # 실행 계획을 볼 수 있다
select count(*)
from salaries
where to_date > "2000-01-01"

create index tdate
on salaries(to_date)

drop index tdate # 인덱스 삭제
on salaries

# 인덱스 너무 많이 쓰면 안 좋다
# 인덱스를 탄 쿼리는 빨라지지지만
# 인덱스 사용하면 저장공간 더 많이 사용하게 된다
# 데이터베이스 추가 될 때마다 인덱스를 업데이트해줘야 한다, 인서트가 느려진다
# 보통 where 절에 사용된 컬럼을 인덱스로 많이 지정해 준다

# view : 특정 쿼리 문에 대한 결과를 저장하는 개념
use world

create view code_name as
select code, name
from country

select *
from city
join code_name
# (select code, name
# from country) as code_name 코드를 줄인 것
on city.countrycode = code_name.code

# 뷰는 수정이나 인덱스 설정하는 게 아니다. 직접 테이블이 생성돼서 데이터를 복사해서 갖고 있는 것도 아냐
# 쿼리만 간단하게 만들어주는 것